In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
np.set_printoptions(precision=3, suppress=True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
### Khách hàng thường mua bao nhiêu sản phẩm cho 1 đơn hàng

In [111]:
file_path = 'data/order_products__train.csv'
df = pd.read_csv(file_path)
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [113]:
orders = df['order_id'].value_counts().reset_index()

orders.columns = ['order_id','counts']
orders

In [135]:
fig = px.histogram(orders, x='counts', nbins=80, color_discrete_sequence=['indianred'], opacity=0.75)
fig.show()

### Sản phẩm bán chạy nhất

In [137]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [140]:
top10_product = df['product_id'].value_counts().reset_index()
top10_product.columns = ['product_id','counts']
top10_product = top10_product.sort_values(by=['counts'], ascending=False)[:10]
top10_product

,product_id,counts
0,24852,18726
1,13176,15480
2,21137,10894
3,21903,9784
4,47626,8135
5,47766,7409
6,47209,7293
7,16797,6494
8,26209,6033
9,27966,5546


In [ ]:
# Load dữ liệu lấy tên sản phẩm
products = pd.read_csv('data/products.csv')

# Merge 2 bảng để lấy ra tên top 10 sản phẩm.
product_name = pd.merge(products, top10_product, on='product_id')[['product_name','counts']]

# Sắp xếp lại dữ liệu
product_name = product_name.sort_values(by='counts', ascending=False)


In [170]:
fig = go.Figure(layout=go.Layout(xaxis={'type': 'category'}, title='Top 10 sản phẩm bán chạy nhất'))
goBar = go.Bar(x=product_name['product_name'], y=top10_product['counts'])
fig.add_trace(goBar)

### Sản phẩm nào được re-ordered nhiều nhất.

In [152]:
reordered=df[df['reordered']==1]

In [158]:
counts = reordered['product_id'].value_counts().reset_index()
counts.columns = ['product_id','count']
counts = counts.sort_values(by=['count'], ascending=False)[:10]
counts

,product_id,count
0,24852,16557
1,13176,13362
2,21137,8603
3,21903,8055
4,47766,6226
5,47209,6042
6,47626,5923
7,16797,4786
8,27966,4279
9,26209,4234


In [165]:
counts_name = pd.merge(counts, products, on='product_id')[['product_id','product_name','count']]
counts_name = counts_name.sort_values(by='count', ascending=False)
counts_name

,product_id,product_name,count
0,24852,Banana,16557
1,13176,Bag of Organic Bananas,13362
2,21137,Organic Strawberries,8603
3,21903,Organic Baby Spinach,8055
4,47766,Organic Avocado,6226
5,47209,Organic Hass Avocado,6042
6,47626,Large Lemon,5923
7,16797,Strawberries,4786
8,27966,Organic Raspberries,4279
9,26209,Limes,4234


In [169]:
fig = go.Figure(layout=go.Layout(xaxis={'type':'category'}, title='Top 10 sản phẩm được reorder nhiều nhất'))
fig.add_trace(go.Bar(x=counts_name['product_name'], y=counts_name['count']))
fig.show()

### Kệ hàng nào bán được nhiều sản phẩm nhất

In [2]:
# Load dữ liệu
file_path = 'demo2_data/data/products.csv'
product_info = pd.read_csv(file_path)
product_info.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [70]:
# Làm việc với dữ liệu lớn với chunksize
## Tổng quan dữ liệu order_product__prior.csv
N = 5e5
file_path = 'demo2_data/data/order_products__prior.csv'
order_products = pd.read_csv(file_path, chunksize=N)
chunk = next(order_products)
chunk.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [71]:
chunk.shape

(500000, 4)

In [72]:
# Hàm thực hiện merge dữ liệu
def Select_Aisle_ID(order_pro_id):
    product_aisle = product_info[['product_id', 'aisle_id']]
    order_aisle = pd.merge(order_pro_id, product_aisle, on='product_id')['aisle_id']
    aisle_count = order_aisle.value_counts().reset_index()
    aisle_count.columns = ['aisle_id', 'counts']
    return aisle_count

total_aisle_count = Select_Aisle_ID(chunk)

# Thực hiện ghép các chunk dữ liệu
for i, chunk in enumerate(order_products):
    if i > 65: break
    chunk_count = Select_Aisle_ID(chunk)
    total_aisle_count = pd.concat([total_aisle_count, chunk_count])

print(total_aisle_count.shape)
print(total_aisle_count.head())

(8710, 2)
   aisle_id  counts
0        24   56326
1        83   52073
2       123   27347
3       120   22485
4        21   14960


In [73]:
# Truy vấn dữ liệu cần 
total_aisle_count = total_aisle_count.groupby('aisle_id').sum().reset_index()
total_aisle_count.head()

,aisle_id,counts
0,1,71928
1,2,82491
2,3,456386
3,4,200687
4,5,62510


In [74]:
# Lấy ra 10 kệ hàng bán được nhiều hàng nhất
total_aisle_count = total_aisle_count.groupby('aisle_id').sum().sort_values(by=['counts'], ascending=False).reset_index()[:10]
total_aisle_count

,aisle_id,counts
0,24,3642188
1,83,3418021
2,123,1765313
3,120,1452343
4,21,979763
5,84,891015
6,115,841533
7,107,722470
8,91,638253
9,112,584834


In [172]:
# Load dữ liệu tên các kệ hàng
aisle = pd.read_csv('data/aisles.csv')
aisle_name = pd.merge(total_aisle_count, aisle, on='aisle_id')
aisle_name = aisle_name.sort_values(by='counts', ascending=False)

In [173]:
# Vẽ biểu đồ
fig = go.Figure(layout=go.Layout(xaxis={'type': 'category'}, title='Top 10 kệ hàng được mua nhiều nhất'))
goBar = go.Bar(x=aisle_name['aisle'], y=aisle_name['counts'])
fig.add_trace(goBar)